In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
#Download training data from open database
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 21.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 340kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 6.32MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.0MB/s]


In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

#create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)

for X,y in test_dataloader:
  print(f"Shape of X [N,C,H,W] : {X.shape}")
  print(f"Shape of Y : {y.shape} {y.dtype}")
  break


Shape of X [N,C,H,W] : torch.Size([64, 1, 28, 28])
Shape of Y : torch.Size([64]) torch.int64


In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Defin model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

Using cpu device


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [3]:
tensor3d = torch.tensor([[[1,2],[3,4]], [[5,6],[7,8]],[[9,10],[11,12]]])

floattensor = tensor3d.to(torch.float32)

In [6]:
tensor3d

tensor([[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 9, 10],
         [11, 12]]])

In [7]:
tensor2d = torch.tensor([[1,2],[3,4], [5,6],[7,8],[9,10],[11,12]])

In [11]:
tensor1d = tensor2d.flatten()
tensor1d

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [9]:
tensor2d.T.matmul(tensor2d)

tensor([[286, 322],
        [322, 364]])

In [11]:
import torch.nn.functional as F
from torch.autograd import grad

In [12]:
y = torch.tensor([1.0]) # true label
x1 = torch.tensor([1.1])  # input feature
w1 = torch.tensor([2.2],requires_grad=True) # weight parameter
b = torch.tensor([0.0],requires_grad=True)
z = x1*w1 +b # net input
a = torch.sigmoid(z) #activation and output
loss = F.binary_cross_entropy(a,y)

grad_L_w1 = grad(loss, w1, retain_graph = True)
grad_L_b = grad(loss, b, retain_graph = True)
print(loss)
print(grad_L_w1)
print(grad_L_b)

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)
(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [13]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [31]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super().__init__()
    self.layers = torch.nn.Sequential(
        # 1st hidden layer
        torch.nn.Linear(num_inputs,30),
        torch.nn.ReLU(),
         # 2nd hidder layer
        torch.nn.Linear(30,20),
        torch.nn.ReLU(),
        # output layer
        torch.nn.Linear(20, num_outputs),
    )
  def forward(self,x):
    logits = self.layers(x)
    return logits

In [32]:
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [17]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad )

In [18]:
print("Total number of trainable model parameters : ", num_params)

Total number of trainable model parameters :  2213


In [34]:
torch.manual_seed(123)
print(model.layers[4].weight)

Parameter containing:
tensor([[ 0.2153,  0.1073, -0.1316,  0.1673, -0.2174,  0.1197,  0.1110,  0.1149,
          0.0641,  0.1838, -0.1232,  0.1640,  0.0877,  0.0059,  0.0885, -0.0207,
          0.1805,  0.0039,  0.1246,  0.1993],
        [ 0.0725,  0.0062, -0.0157, -0.1822, -0.1747,  0.0746,  0.0208,  0.0579,
         -0.2020,  0.1252, -0.0231,  0.0871, -0.1233,  0.0467,  0.1151, -0.1672,
          0.0199, -0.0774,  0.1836, -0.0468],
        [-0.1308, -0.1433, -0.0204,  0.1016,  0.0757,  0.2032,  0.1731,  0.0369,
          0.0617, -0.0968,  0.0785,  0.1716, -0.0046,  0.0431, -0.1838,  0.1254,
          0.1889,  0.2059,  0.0939, -0.0861]], requires_grad=True)


In [35]:
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[ 0.0670, -0.1420, -0.1203]], grad_fn=<AddmmBackward0>)


In [36]:
with torch.no_grad():
  out  = model(X)
print(out)

tensor([[ 0.0670, -0.1420, -0.1203]])


In [ ]:
from torch.utils.data import Dataset
class ToyDataset(Dataset):
  def __init__(self,X,y):
    self.features = X
    self.labels = y
  def __getitem__(self, index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y
  def __len__(self):
    return self.labels.shape[0]
train_ds  = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)